In [1]:
import numpy as np
from collections import defaultdict
import pandas as pd

In [15]:
voting_data = [
        [5, ['a', 'b', 'c', 'd']],
        [4, ['a', 'c', 'b', 'd']],
        [2, ['d', 'b', 'a', 'c']],
        [6, ['d', 'b', 'c', 'a']],
        [8, ['c', 'b', 'a', 'd']],
        [2, ['d', 'c', 'b', 'a']]
    ]

In [16]:
def plurality(voting_data):
    vote_counts = {}
    for candidate, votes in voting_data.items():
        if candidate in vote_counts:
            vote_counts[candidate] += votes
        else:
            vote_counts[candidate] = votes
    return max(vote_counts, key=vote_counts.get)

voting_data = {'a': 9, 'd': 10, 'c': 8}
winner = plurality(voting_data)
print(f"The plurality winner is: {winner}")

The plurality winner is: d


In [17]:
def PluralityRunoff(voting_data):
    total_votes = sum(voting_data.values())
    first_place_counts = {}

    for preference, count in voting_data.items():
        first_choice = preference.split(' ≻ ')[0]
        if first_choice in first_place_counts:
            first_place_counts[first_choice] += count
        else:
            first_place_counts[first_choice] = count

    for candidate, count in first_place_counts.items():
        if count > total_votes / 2:
            return candidate

    if len(first_place_counts) > 1:  
        top_two = sorted(first_place_counts, key=first_place_counts.get, reverse=True)[:2]
        
        runoff_counts = {candidate: 0 for candidate in top_two}
        for preference, count in voting_data.items():
            first_choice = preference.split(' ≻ ')[0]
            if first_choice in runoff_counts:
                runoff_counts[first_choice] += count
                
        return max(runoff_counts, key=runoff_counts.get)

winner = PluralityRunoff(voting_data)
print(f"The winner of the plurality runoff vote is: {winner}")

The winner of the plurality runoff vote is: d


In [21]:
def CondorcetVoting(voting_data):
    # Extract candidates and initialize win count for pairwise comparisons
    candidates = set()
    for preference in voting_data:
        candidates.update(preference.split(' ≻ '))
    wins = {candidate: {other: 0 for other in candidates if other != candidate} for candidate in candidates}

    # Count pairwise wins
    for preference, count in voting_data.items():
        ordered_candidates = preference.split(' ≻ ')
        for i, winner in enumerate(ordered_candidates):
            for loser in ordered_candidates[i+1:]:
                wins[winner][loser] += count

    # Determine if there's a Condorcet winner
    condorcet_winner = None
    for candidate in candidates:
        if all(wins[candidate][opponent] > sum(voting_data.values()) / 2 for opponent in wins[candidate]):
            condorcet_winner = candidate
            break

    return condorcet_winner

winner = CondorcetVoting(voting_data)
if winner:
    print(f"The Condorcet winner is: {winner}")
else:
    print("There is no Condorcet winner.")

There is no Condorcet winner.


In [1]:
import pandas as pd
from collections import Counter

# Data from the example
voting_data = {
    'a ≻ b ≻ c ≻ d': 5,
    'a ≻ c ≻ b ≻ d': 4,
    'd ≻ b ≻ a ≻ c': 2,
    'd ≻ b ≻ c ≻ a': 6,
    'c ≻ b ≻ a ≻ d': 8,
    'd ≻ c ≻ b ≻ a': 2
}

# Transform data into a more manageable format
def expand_voting_data(voting_data):
    expanded_list = []
    for preference, count in voting_data.items():
        for _ in range(count):
            expanded_list.append(preference.split(' ≻ '))
    return expanded_list

# Convert the list to DataFrame for easier manipulation
def create_voter_df(expanded_list):
    return pd.DataFrame(expanded_list, columns=['1st', '2nd', '3rd', '4th'])

voter_df = create_voter_df(expand_voting_data(voting_data))

# Implement Plurality Voting
def plurality(voter_df):
    return voter_df['1st'].value_counts().idxmax()

# Implement Plurality with Runoff
def runoff(voter_df):
    first_round = voter_df['1st'].value_counts()
    if first_round.max() > len(voter_df) / 2:
        return first_round.idxmax()
    else:
        # Runoff between top two
        top_two = first_round.nlargest(2).index
        second_round = voter_df[voter_df['1st'].isin(top_two)]['1st'].value_counts()
        return second_round.idxmax()

# Implement Condorcet Method
def condorcet(voter_df):
    candidates = voter_df['1st'].unique()
    wins = {candidate: 0 for candidate in candidates}
    for candidate in candidates:
        for opponent in candidates:
            if candidate != opponent:
                # Win if candidate beats opponent in more voter preferences
                wins[candidate] += int(sum(voter_df.apply(lambda row: row.tolist().index(candidate) < row.tolist().index(opponent), axis=1)) > len(voter_df) / 2)
    # Candidate who beats all others
    for candidate, win_count in wins.items():
        if win_count == len(candidates) - 1:
            return candidate
    return None  # No Condorcet winner

# Implement Borda Count
def borda(voter_df):
    candidates = voter_df['1st'].unique()
    scores = {candidate: 0 for candidate in candidates}
    positions = ['1st', '2nd', '3rd', '4th']
    for position in positions:
        for candidate in candidates:
            scores[candidate] += sum((4 - positions.index(position)) * (voter_df[position] == candidate))
    return max(scores, key=scores.get)

# Execute the voting methods
print("Plurality Winner:", plurality(voter_df))
print("Runoff Winner:", runoff(voter_df))
print("Condorcet Winner:", condorcet(voter_df))
print("Borda Winner:", borda(voter_df))

Plurality Winner: d
Runoff Winner: d
Condorcet Winner: c
Borda Winner: c
